In [2]:
import numpy as np
import cv2
#import pandas as pd
import os
import tifffile as tiff
from PIL import Image
import skimage
import czifile

In [4]:
def czi_to_tiff(czi_folder,save_path):
    """read uint16 czi files and save as uint8 tiff files"""
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for root, dirs, files in os.walk(czi_folder):
        for file in files:
            if file.endswith('.czi'):
                print(file)
                czi = czifile.CziFile(os.path.join(root,file))
                image = czi.asarray()
                image = image.squeeze()
                image = np.moveaxis(image,0,-1)
                #image = skimage.exposure.rescale_intensity(image, in_range='dtype', out_range='uint8')
                tiff.imsave(os.path.join(save_path,file.replace('.czi','.tif')),image)

czi_to_tiff('/Users/tom/Downloads/BZ_OL_images','/Users/tom/Downloads/BZ_OL_images_tiff')


BZ-MBL_002_20x_LatA500nM-right-19.czi
BZ-MBL_002_20x_DMSO-left-02_photogenic.czi
BZ-MBL_002_20x_LatA2uM-left-23.czi


/var/folders/kh/yrhstrqj6171b88zc_cvt6hm0000gn/T/ipykernel_68472/1459103529.py:14: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tiff.imsave(os.path.join(save_path,file.replace('.czi','.tif')),image)


BZ-MBL_002_20x_LatA500nM-right-18.czi
BZ-MBL_002_20x_ionomycin-left-40.czi
BZ-MBL_002_20x_LatA2uM-right-29.czi
BZ-MBL_002_20x_LatA2uM-left-24.czi
BZ-MBL_002_20x_LatA2uM-right-28.czi
BZ-MBL_002_20x_LatA2uM-left-25.czi
BZ-MBL_002_20x_LatA500nM-right-22.czi
BZ-MBL_002_20x_ionomycin-left-37_photogenic.czi
BZ-MBL_002_20x_LatA500nM-right-20.czi
BZ-MBL_002_20x_LatA2uM-left-27.czi
BZ-MBL_002_20x_LatA2uM-left-26.czi
BZ-MBL_002_20x_LatA500nM-right-21.czi
BZ-MBL_002_20x_ionomycin-left-34.czi
BZ-MBL_002_20x_DMSO-left-03.czi
BZ-MBL_002_20x_ionomycin-left-35.czi
BZ-MBL_002_20x_ionomycin-left-36.czi
BZ-MBL_002_20x_DMSO-left-04.czi
BZ-MBL_002_20x_DMSO-right-10.czi
BZ-MBL_002_20x_DMSO-right-11.czi
BZ-MBL_002_20x_DMSO-left-05.czi
BZ-MBL_002_20x_ionomycin-left-33_photogenic.czi
BZ-MBL_002_20x_DMSO-right-07.czi
BZ-MBL_002_20x_DMSO-left-06.czi
BZ-MBL_002_20x_ionomycin-right-46.czi
BZ-MBL_002_20x_DMSO-right-08.czi
BZ-MBL_002_20x_ionomycin-right-45.czi
BZ-MBL_002_20x_DMSO-right-09.czi
BZ-MBL_002_20x_ionomyci

In [5]:
def normalize_img(img, num_channels=3):
    """normalizes each channel of the image by dividing by the maximum value"""
    normalized_channels = []
    max_channels = []
    if num_channels == 1:
        np.expand_dims(img, axis=-1)
    for i in range(num_channels):
        channel = img[:, :,i].astype(np.float32)
        max_channels.append(np.max(channel))
        if max_channels[-1] != 0:
            normalized_channel = channel / max_channels[-1]
        else:
            normalized_channel = channel
        normalized_channels.append(normalized_channel)
    
    return np.stack([channel * 255 for channel in normalized_channels], axis=-1).astype(np.uint8).squeeze(), max_channels 

In [8]:
def convert_to_grayscale(dir_path, save_path):
    """converts multi-channel image to grayscale"""
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for file in os.listdir(dir_path):
        if file.endswith('.tif'):
            img = skimage.io.imread(os.path.join(dir_path, file), as_gray=True)
            print(img)
            skimage.io.imsave(os.path.join(save_path, file), img)

In [21]:
if __name__ == '__main__':
    folder_path = '/Users/tom/Downloads/tifs_from_ome'
    save_path_norm = '/Users/tom/Downloads/normtifs'
    save_path_gray = '/Users/tom/Downloads/graytifs'
    if not os.path.exists(save_path_norm):
        os.makedirs(save_path_norm)
    channel_max = []
    for file in os.listdir(folder_path):
        if file.endswith('.tif'):
            print(os.path.basename(file))
            img = skimage.io.imread(os.path.join(folder_path, file)) # better for handling more than 3 channels.
            assert img.shape[-1] < 30, 'Check in which index position are stored the channels.'
            img, max_channels = normalize_img(img, num_channels=img.shape[-1])
            skimage.io.imsave(os.path.join(save_path_norm, os.path.basename(file)), img)
            channel_max.append([os.path.basename(file)] + max_channels)
    
    header = 'file,' + ','.join([f'c{i+1}_max' for i in range(len(max_channels))]) + '\n'
    with open('channel_max.csv', 'w') as f:
        f.write(header)
        for row in channel_max:
            f.write(','.join(map(str, row)) + '\n')

    convert_to_grayscale(save_path_norm, save_path_gray)
    print(f'{len(os.listdir(save_path_norm))} images were normalized and saved as grayscale.')

CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_12.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_10.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_11.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_8.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_9.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_7.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_6.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_4.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_5.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_1.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_0.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_2.tif
CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigne

In [6]:
if __name__ == '__main__':
    folder_path = '/Users/tom/Downloads/BZ_OL_images_tiff'
    save_path_norm = '/Users/tom/Downloads/BZ_OL_images_tiff_norm'
    save_path_gray = '/Users/tom/Downloads/graytifs'
    if not os.path.exists(save_path_norm):
        os.makedirs(save_path_norm)
    channel_max = []
    for file in os.listdir(folder_path):
        if file.endswith('.tif'):
            print(os.path.basename(file))
            img = skimage.io.imread(os.path.join(folder_path, file),as_gray=True)
            img = skimage.exposure.equalize_adapthist(img, clip_limit=0.02)
            skimage.io.imsave(os.path.join(save_path_norm, os.path.basename(file)), img)
    print(f'{len(os.listdir(save_path_norm))} images were normalized and saved as grayscale.')

BZ-MBL_002_20x_LatA2uM-right-28.tif
BZ-MBL_002_20x_LatA2uM-left-25.tif
BZ-MBL_002_20x_ionomycin-left-37_photogenic.tif
BZ-MBL_002_20x_LatA500nM-right-22.tif
BZ-MBL_002_20x_LatA2uM-right-29.tif
BZ-MBL_002_20x_LatA2uM-left-24.tif
BZ-MBL_002_20x_LatA2uM-left-26.tif
BZ-MBL_002_20x_LatA500nM-right-21.tif
BZ-MBL_002_20x_LatA500nM-right-20.tif
BZ-MBL_002_20x_LatA2uM-left-27.tif
BZ-MBL_002_20x_LatA2uM-left-23.tif
BZ-MBL_002_20x_LatA500nM-right-18.tif
BZ-MBL_002_20x_LatA500nM-right-19.tif
BZ-MBL_002_20x_DMSO-left-02_photogenic.tif
BZ-MBL_002_20x_ionomycin-left-40.tif
BZ-MBL_002_20x_DMSO-right-11.tif
BZ-MBL_002_20x_DMSO-left-05.tif
BZ-MBL_002_20x_DMSO-left-04.tif
BZ-MBL_002_20x_DMSO-right-10.tif
BZ-MBL_002_20x_DMSO-left-06.tif
BZ-MBL_002_20x_ionomycin-left-33_photogenic.tif
BZ-MBL_002_20x_DMSO-right-07.tif
BZ-MBL_002_20x_DMSO-left-03.tif
BZ-MBL_002_20x_ionomycin-left-35.tif
BZ-MBL_002_20x_ionomycin-left-34.tif
BZ-MBL_002_20x_ionomycin-left-36.tif
BZ-MBL_002_20x_ionomycin-right-41.tif
BZ-MBL_002_

In [13]:
def ometifs_to_tifs(dir_path, save_path):
    """converts ome-tifs to tifs"""
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for file in os.listdir(dir_path):
        if file.endswith('.ome.tif'):
            img = skimage.io.imread(os.path.join(dir_path, file))
            for i in range(img.shape[0]):
                skimage.io.imsave(os.path.join(save_path, file[:-8] + f'_{i}.tif'), img[i])

In [14]:
ometifs_to_tifs('/Users/tom/Downloads/ometifs', '/Users/tom/Downloads/tifs_from_ome')

/var/folders/kh/yrhstrqj6171b88zc_cvt6hm0000gn/T/ipykernel_48740/770908100.py:9: UserWarning: /Users/tom/Downloads/tifs_from_ome/CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_0.tif is a low contrast image
  skimage.io.imsave(os.path.join(save_path, file[:-8] + f'_{i}.tif'), img[i])
/var/folders/kh/yrhstrqj6171b88zc_cvt6hm0000gn/T/ipykernel_48740/770908100.py:9: UserWarning: /Users/tom/Downloads/tifs_from_ome/CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_1.tif is a low contrast image
  skimage.io.imsave(os.path.join(save_path, file[:-8] + f'_{i}.tif'), img[i])
/var/folders/kh/yrhstrqj6171b88zc_cvt6hm0000gn/T/ipykernel_48740/770908100.py:9: UserWarning: /Users/tom/Downloads/tifs_from_ome/CE031_I1_div3_1hrinterval_concatenated_tx_aftertp1_timealigned_scP99-B3_2.tif is a low contrast image
  skimage.io.imsave(os.path.join(save_path, file[:-8] + f'_{i}.tif'), img[i])
/var/folders/kh/yrhstrqj6171b88zc_cvt6hm0000gn/T/ipykernel_48740/77090810